In [1]:
# 데이터 전처리하기

[1] 모듈, 데이터 불러오기<hr>

In [2]:
from Korpora import Korpora


import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [3]:
# 데이터 불러오기
nsmc= Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-25\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [4]:
# test 데이터셋만 전처리
Data= nsmc.test
DataDF=pd.DataFrame(Data)

[2] 텍스트 데이터 전처리 <hr>
- 토큰화/정제 (불용어, 구두점, 오타 등 처리)
- 단어사전 생성
- 문장 수치화

In [5]:
# 단어 사전을 생성시 활용할 데이터셋

class TextDataset(Dataset):
    def __init__(self, feature, label):
        super().__init__()
        self.feature= feature
        self.label= label
        self.length=feature.shape[0]

    
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.feature[index], self.label[index]

In [6]:
# DS 인스턴스 생성
nsmcDS=TextDataset(DataDF['text'], DataDF['label'])

In [7]:
for f, l in nsmcDS:
    print(f,l)
    break

굳 ㅋ 1


In [31]:
from konlpy.tag import *
import string
from nltk.tokenize import word_tokenize
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
# 구두점, 특수문자 추출
pun=list(string.punctuation)

In [10]:
# 토큰화 인스턴스 생성
okt= Okt()

In [24]:
tokens=okt.pos(DataDF['text'][0], stem=True)

In [25]:
tokens

[('굳다', 'Adjective'), ('ㅋ', 'KoreanParticle')]

In [28]:
# 토큰화를 통한 단어사전 생성 ->Counter와 리스트 컴프리헨션을 통해
# - 행마다 데이터를 추출하여 토큰 반환 및 Counter에 저장
def make_vocab(dataset):
    '''
    DF 형태일 경우 인덱스 초기화하기!
    '''
    counter=Counter()
    for text,label in dataset:
        # 한글빼고 다지우기
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)
        # 형태소 분석 (stem-> 어근으로 통일, norm-> 정규화)
        tokens=okt.pos(text,norm=True,stem=True)
        #불용어, 구두점, 특수문자 제거


        # 형태소 단어 counter에 저장
        for t in tokens:
            counter.update(t)
    # 단어 사전에 저장
    vocab={'<PAD>':0, '<UNK>':1}
    vocab.update({word: i+2 for i, word in enumerate(counter.items())})
    return vocab

In [ ]:
# 토큰화를 통한 단어사전 생성 ->CountVectorize를 통해
# - 행마다 데이터를 추출하여 리스트에 저장 -> 단어 사전 생성성
def make_vocab(dataset):
    tokenlists=[]
    counter=Counter()
    for text,label in dataset:
        # 한글빼고 다지우기
        text=re.sub('[^ㄱ-ㅎ가-힣]+',' ',text)
        # 형태소 분석 (stem-> 어근으로 통일, norm-> 정규화)
        tokens=okt.morphs(text)
        #불용어, 구두점, 특수문자 제거


        # 형태소 단어 tokenlists에 저장
        tokenlists.append(tokens)
    # 단어 사전 생성
    cv=CountVectorizer()
    cv.fit_transform(tokenlists)
    vocab=cv.vocabulary_


In [ ]:
# 단어사전을 통해 문장 벡터화 진행
# 받은 텍스트를 같은 방법으로 토큰화 -> 만들어진 단어사전과 대조하여 수치화
def vectorize(vocab, text, tokenizer):
    vector_list=[]
    for t in text:
        token_lists=tokenizer.pos(t)
        vector_token=[vocab[token] if token in vocab else vocab['<UNK>'] for token in token_lists]
        vector_list.append(vector_token)

In [ ]:
# 수치화 패딩
def vectorize(tokenlist,padding=20):
    ### 단어 사전 생성 및 변환
    cv=CountVectorizer()
    cv.fit_transform(tokenlist)
    vo=cv.vocabulary_
    vectorlist=tokenlist.copy()
    for idx,sen in enumerate(vectorlist):
        senlist=sen.split(' ')[1:]
        length=len(senlist)
        if length<padding:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist+([0]*(padding-length))
        else:
            for ind,st in enumerate(senlist):
                senlist[ind]=vo[st]
            vectorlist[idx]=senlist[:padding]

    return vectorlist

In [12]:
# 패딩
# 1. 선택한 길이가 문장길이보다 긴 경우 -> 문장 끝에 0추가
# 2. 선택한 길이가 문장길이보다 짧은 경우 -> 패딩길이만큼 슬라이싱
# ==> 일반적으로 한글은 중요한 정보가 앞쪽에 있다. -> 뒤쪽에 패딩추가 또는 슬라이싱
def padding(length, textList):
    pad_texts=[]
    for text in textList:
            # 선택 길이> 문장길이 일때
        if length>len(text):
                                            # 남은 텍스트 길이만큼 0으로 채우기
            text=text+[0]*(length-text)
        else: # 선택 길이< 문장 길이 일때
            text=text[:length]              # 선택 길이만큼 슬라이싱
        pad_texts.append(text)

KeyError: 50000